import

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import torch
import torch.nn as nn
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from matplotlib import gridspec
import pickle as pkl
import itertools
import matplotlib.pyplot as plt
import sys, os
opj = os.path.join
ope = os.path.exists
this_dir = os.getcwd()
lib_paths = [opj(this_dir, 'bestfitting/protein_clean/src'), opj(this_dir, 'CD'), opj(this_dir, 'viz')]
for lib_path in lib_paths:
    if lib_path not in sys.path:
        sys.path.insert(0, lib_path) 

In [2]:
from cd_propagate import *
from cd import *
from dset import *
from score_funcs import *
import viz 
from config.config import * # set directory paths (DATA_DIR, RESULT_DIR etc) 
import train_cls_net # import Protein class
from net.loss_funcs.kaggle_metric import prob_to_result # import prob_to_result
from net.loss_funcs.kaggle_metric import get_probs_f1_score # import get_probs_f1_score  
import warnings
warnings.filterwarnings("ignore")

run on ip-172-31-63-74


### Set parameters and directories

In [3]:
# get args
args = get_args()

# set dirs
# fname=which file to evaluate
dir_args, data_args = set_dirs(args, fname='valid_160.csv')

# get Protein class
trainer = train_cls_net.Protein(dir_args,
                                train_batch_size=args.train_batch_size,
                                test_batch_size=args.test_batch_size,
                                seed=args.seed, img_size=args.img_size,in_channels=args.in_channels,
                                save_probs=args.save_probs,
                                aug_version=args.aug_version,
                                num_classes=args.num_classes,
                                crop_size=args.crop_size,
                                clipnorm=args.clipnorm)

# load model
net, scheduler, loss = load_densenet_pretrained(args, trainer, data_args)

# test dataset and dataloader
test_dataset = get_dataset(trainer)
test_loader = protein_dataset.DataLoader(test_dataset,
                                         sampler=SequentialSampler(test_dataset),
                                         batch_size=trainer.test_batch_size,
                                         drop_last=False,
                                         num_workers=trainer.num_workers,
                                         pin_memory=True)

in_channels 4
load model file from: /home/ubuntu/HPA/hpa_interp/bestfitting/protein_clean/result/models/external_crop1024_focal_slov_hardlog_clean_class_densenet121_large_dropout_i1536_aug2_5folds/fold0/final.pth
using gpu 0


RuntimeError: CUDA error: out of memory

In [ ]:
# use gpu
if trainer.gpu_flag:
    net.cuda()
# net eval mode
net.eval()

print('ready to evaluate')

### CD score

In [ ]:
from cd_propagate import *
# load test image
load_img = pkl.load(open('./data/test_image', 'rb'))
test_image = load_img['image']
test_label = load_img['label']

# viz test image
img_index = 0
viz.viz_channels_separate(test_image, img_index)
print(test_label)

In [ ]:
# test image
im = test_image.clone()

# switch dtype
dtype = torch.float
im = torch.tensor(im, dtype=dtype)
net.float() if dtype==torch.float else net.double()
print('switch to', dtype)

# gradCAM
cam, target = grad_CAM(im, net, layer_ind=11)

In [ ]:
viz.viz_channels_combined(test_image, img_index, cam)